# Анализ датасета Titanic

## Описание задания

В этом проекте проводится полный анализ известного датасета "Титаник". Задание включает:
1. **Подготовку данных** - загрузку, объединение и приведение типов
2. **Анализ таблицы** - статистику, группировки, фильтрации
3. **Визуализацию** - построение 10 различных графиков

## Описание датасета

Датасет Titanic содержит информацию о пассажирах знаменитого корабля:
- **PassengerId** - идентификатор пассажира
- **Survived** - выжил (1) или нет (0)
- **Pclass** - класс каюты (1, 2, 3)
- **Name** - имя пассажира
- **Sex** - пол
- **Age** - возраст
- **SibSp** - количество братьев/сестер/супругов на борту
- **Parch** - количество родителей/детей на борту
- **Ticket** - номер билета
- **Fare** - стоимость билета
- **Cabin** - номер каюты
- **Embarked** - порт посадки (C = Cherbourg, Q = Queenstown, S = Southampton)

# Ход работы

## 1. Установка библиотек

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 2. Подготовка данных

### 2.1 Установка и оформление датасета
#### 2.1.1 Установка датасета
```python
train_frame = pd.read_csv('../../datasets/ha_7/train.csv')
test_frame = pd.read_csv('../../datasets/ha_7/test.csv')
```
#### 2.1.2 Конкатенация
```python
test_frame['Survived'] = None
dataframe = pd.concat([train_frame, test_frame], ignore_index=True, sort=False)
```
#### 2.1.3 Категоризация переменных
```python
dataframe['Survived'] = dataframe['Survived'].astype('category')
dataframe['Pclass'] = dataframe['Pclass'].astype('category')
dataframe['Sex'] = dataframe['Sex'].astype('category')
```

#### **ИСПРАВЛЕНИЕ**: добавил явную конкатенацию по PassengerID
```python
dataframe = pd.concat([train_frame, test_frame], ignore_index=True)
dataframe = dataframe.sort_values('PassengerId').reset_index(drop=True)
```

#### **ИСПРАВЛЕНИЕ**: Добавил преобразование всех столбцов в подхоящие типы
```python
categorial_columns = ['Embarked', 'Survived', 'Pclass', 'Sex']
for col in categorial_columns:
  if col in dataframe.columns:
      dataframe[col] = dataframe[col].astype('category')

text_columns = ['Name', 'Ticket', 'Cabin']
for col in text_columns:
  if col in dataframe.columns:
      dataframe[col] = dataframe[col].astype('string')
    
numeric_columns = ['Age', 'Fare', 'SibSp', 'Parch']
for col in numeric_columns:
    if col in dataframe.columns:
        dataframe[col] = pd.to_numeric(dataframe[col], errors='coerce')
    
dataframe['PassengerId'] = dataframe['PassengerId'].astype('int64')
```

In [112]:
def load_titanic():
    train_frame = pd.read_csv('../../datasets/ha_7/train.csv')
    test_frame = pd.read_csv('../../datasets/ha_7/test.csv')
    
    test_frame['Survived'] = None

    dataframe = pd.concat([train_frame, test_frame], ignore_index=True)
    dataframe = dataframe.sort_values('PassengerId').reset_index(drop=True)

    categorial_columns = ['Embarked', 'Survived', 'Pclass', 'Sex']
    for col in categorial_columns:
      if col in dataframe.columns:
        dataframe[col] = dataframe[col].astype('category')

    text_columns = ['Name', 'Ticket', 'Cabin']
    for col in text_columns:
        if col in dataframe.columns:
            dataframe[col] = dataframe[col].astype('string')
    
    numeric_columns = ['Age', 'Fare', 'SibSp', 'Parch']
    for col in numeric_columns:
        if col in dataframe.columns:
            dataframe[col] = pd.to_numeric(dataframe[col], errors='coerce')
    
    dataframe['PassengerId'] = dataframe['PassengerId'].astype('int64')

    return dataframe

dataframe = load_titanic()

## 3. Анализ таблицы

### 3.1 Базовая статистика таблицы

In [91]:
def basic_statistics(dataframe: pd.DataFrame):
    print("Базовая статистика таблицы:")
    print(f"Размер таблицы: {dataframe.shape[0]} строк, {dataframe.shape[1]} столбцов")
    print(f"Типы данных:")
    print(dataframe.dtypes)
    print(f"\nПропущенные значения:")
    print(dataframe.isnull().sum())
    print(f"\nПамять: {dataframe.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

basic_statistics(dataframe)

Базовая статистика таблицы:
Размер таблицы: 1309 строк, 12 столбцов
Типы данных:
PassengerId             int64
Survived             category
Pclass               category
Name           string[python]
Sex                  category
Age                   float64
SibSp                   int64
Parch                   int64
Ticket         string[python]
Fare                  float64
Cabin          string[python]
Embarked             category
dtype: object

Пропущенные значения:
PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

Память: 0.3 MB


### 3.2 Класс с наибольшим количеством пассажиров

#### **ИСПРАВЛЕНИЕ**: убрал повторную загрузку датасета (не заметил после установки предварительной загрузки в прошлой ячейке)
```python
maximum_passengers(dataframe)
```

In [ ]:
def maximum_passengers(dataframe : pd.DataFrame):
    class_counts = dataframe['Pclass'].value_counts()
    print("Количество пассажиров по классам:")
    print(class_counts)
    print(f"Больше всего пассажиров было в {class_counts.idxmax()} классе: {class_counts.max()} человек")

maximum_passengers(dataframe)

Количество пассажиров по классам:
Pclass
3    709
1    323
2    277
Name: count, dtype: int64
Больше всего пассажиров было в 3 классе: 709 человек


### 3.3 Двухуровневая группировка таблицы
#### 3.3.1 Средний возраст по классу и полу
```python
age_by_class_sex = dataframe.groupby(['Pclass', 'Sex'], observed=True)['Age'].mean()
```
#### 3.3.2 Самый юный и самый взрослый
```python
youngest_group = age_by_class_sex.idxmin()
oldest_group = age_by_class_sex.idxmax()
youngest_age = age_by_class_sex.min()
oldest_age = age_by_class_sex.max()
print(f"\nСамый юный: {youngest_group[0]} класс, {youngest_group[1]} - {youngest_age:.1f} лет")
print(f"Самый взрослый: {oldest_group[0]} класс, {oldest_group[1]} - {oldest_age:.1f} лет")
```
#### 3.3.3 Разница в возрасте между самым юным и самым взрослым
```python
 difference = oldest_age - youngest_age
```

In [93]:
def group_by_sex(dataframe: pd.DataFrame):
    age_by_class_sex = dataframe.groupby(['Pclass', 'Sex'], observed=True)['Age'].mean()
    print("\nСредний возраст по классу и полу:")
    print(age_by_class_sex)

    youngest_group = age_by_class_sex.idxmin()
    oldest_group = age_by_class_sex.idxmax()
    youngest_age = age_by_class_sex.min()
    oldest_age = age_by_class_sex.max()
    difference = oldest_age - youngest_age

    print(f"\nСамый юный: {youngest_group[0]} класс, {youngest_group[1]} - {youngest_age:.1f} лет")
    print(f"Самый взрослый: {oldest_group[0]} класс, {oldest_group[1]} - {oldest_age:.1f} лет")
    print(f"Разница: {difference:.1f} лет")

group_by_sex(load_titanic())


Средний возраст по классу и полу:
Pclass  Sex   
1       female    37.037594
        male      41.029272
2       female    27.499223
        male      30.815380
3       female    22.185329
        male      25.962264
Name: Age, dtype: float64

Самый юный: 3 класс, female - 22.2 лет
Самый взрослый: 1 класс, male - 41.0 лет
Разница: 18.8 лет


### 3.4 Группировка по фамилии
#### 3.4.1 Выжившие с фамилией, начинающейся на 'K'
```python
survived_k = dataframe[
        (dataframe['Survived'] == 1) &
        (dataframe['Name'].str.split(',').str[0].str.startswith('K'))
    ].copy()
```
#### 3.4.2 Сортировка по убыванию стоимости билета
```python
survived_k_sorted = survived_k.sort_values('Fare', ascending=False)
```
#### 3.4.3 Пассажир, заплативший больше всех и меньше всех
```python
max_fare = survived_k_sorted.iloc[0]
min_fare = survived_k_sorted.iloc[-1]
print(f"Самый дорогой билет: {max_fare['Name']} - ${max_fare['Fare']:.2f}")
print(f"Самый дешевый билет: {min_fare['Name']} - ${min_fare['Fare']:.2f}")
```

In [94]:
def group_by_name(dataframe: pd.DataFrame):
    survived_k = dataframe[
        (dataframe['Survived'] == 1) &
        (dataframe['Name'].str.split(',').str[0].str.startswith('K'))
    ].copy()

    survived_k_sorted = survived_k.sort_values('Fare', ascending=False)

    print(f"\nВыжившие с фамилией на К: {len(survived_k_sorted)} человек")
    if not survived_k_sorted.empty:
        max_fare = survived_k_sorted.iloc[0]
        min_fare = survived_k_sorted.iloc[-1]
        print(f"Самый дорогой билет: {max_fare['Name']} - ${max_fare['Fare']:.2f}")
        print(f"Самый дешевый билет: {min_fare['Name']} - ${min_fare['Fare']:.2f}")

group_by_name(load_titanic())


Выжившие с фамилией на К: 9 человек
Самый дорогой билет: Kimball, Mr. Edwin Nelson Jr - $52.55
Самый дешевый билет: Kelly, Miss. Mary - $7.75


### 3.5 Максимальное количество родных у выжившего пассажира
#### 3.5.1 Выжившие с максимальным количеством родных
```python
survived = dataframe[dataframe['Survived'] == 1].copy()
survived['Total_Relatives'] = survived['SibSp'] + survived['Parch']
max_relatives = survived['Total_Relatives'].max()
```

In [95]:
def max_relatives_survived(dataframe: pd.DataFrame):
    survived = dataframe[dataframe['Survived'] == 1].copy()
    survived['Total_Relatives'] = survived['SibSp'] + survived['Parch']
    max_relatives = survived['Total_Relatives'].max()
    
    passengers_with_max = survived[survived['Total_Relatives'] == max_relatives]
    
    print(f"Максимальное количество родных у выжившего пассажира: {max_relatives}")
    print(f"Пассажиры с таким количеством родных:")
    for _, passenger in passengers_with_max.iterrows():
        print(f"- {passenger['Name']}: {passenger['SibSp']} братьев/сестёр/супругов + {passenger['Parch']} родителей/детей")

max_relatives_survived(dataframe)

Максимальное количество родных у выжившего пассажира: 6
Пассажиры с таким количеством родных:
- Asplund, Mrs. Carl Oscar (Selma Augusta Emilia Johansson): 1 братьев/сестёр/супругов + 5 родителей/детей
- Andersson, Miss. Erna Alexandra: 4 братьев/сестёр/супругов + 2 родителей/детей
- Asplund, Miss. Lillian Gertrud: 4 братьев/сестёр/супругов + 2 родителей/детей
- Asplund, Master. Edvin Rojj Felix: 4 братьев/сестёр/супругов + 2 родителей/детей


#### 4. Визуализация

In [96]:
import plotly.io as pio
pio.renderers.default = 'vscode'

### 4.1 Scatter plot

In [97]:
def scatter_plot(dataframe: pd.DataFrame):
    graph = px.scatter(dataframe, x='Age', y='Fare', color = 'Pclass',
                    labels = {'Age':'Passenger age'},
                    hover_data=['Name'], title="Fares by passenger age")
    graph.show()

scatter_plot(load_titanic())

### 4.2 Linear plot
#### 4.2.1 Подсчет шанса выживания и длины имени
```python
dataframe['Name_Length'] = dataframe['Name'].str.len()
dataframe['Survived_Numeric'] = dataframe['Survived'].astype(float)
survival_by_name_length = dataframe.groupby(['Name_Length', 'Sex'], observed=True)['Survived_Numeric'].mean().reset_index()
```

In [98]:
def linear_plot(dataframe: pd.DataFrame):
    dataframe['Name_Length'] = dataframe['Name'].str.len()
    dataframe['Survived_Numeric'] = dataframe['Survived'].astype(float)
    survival_by_name_length = dataframe.groupby(['Name_Length', 'Sex'], observed=True)['Survived_Numeric'].mean().reset_index()
    
    graph = px.line(survival_by_name_length, x='Name_Length', y='Survived_Numeric', color='Sex',
                title='Dependence of survival on the length of the name by sex',
                labels={'Name_Length':'Name length', 'Survived_Numeric':'Survival chance'})
    
    graph.show()

linear_plot(dataframe)

### 4.3 Histogram

In [99]:
def histogram(dataframe: pd.DataFrame):
    dataframe['Ticket length'] = dataframe['Ticket'].str.len()
    graph = px.histogram(dataframe, x='Ticket length', nbins=30,
                    title='Passengers count by specific ticket identifier length')
    graph.show()

histogram(dataframe)

### 4.4 Bar chart
#### 4.1.1 Подсчет средней цены билета
```python
sorted_by_name_length = dataframe.groupby('Name_length')['Fare'].mean().reset_index()
```

In [100]:
def bar_chart(dataframe: pd.DataFrame):
    dataframe['Name_length'] = dataframe['Name'].str.len()
    sorted_by_name_length = dataframe.groupby('Name_length')['Fare'].mean().reset_index()
    sorted_by_name_length = sorted_by_name_length.rename(columns={'Fare': 'Mean_fare'})

    graph = px.bar(sorted_by_name_length, x='Name_length', y='Mean_fare',
                labels={'Name_length':'Passenger full name length'},
                title="Passenger's mean fare by full name length")
    graph.show()

bar_chart(dataframe)

### 4.5 Horizontal bar chart

In [101]:
def horizontal_bar_chart(dataframe: pd.DataFrame):
    dataframe['Ticket_name_length'] = dataframe['Ticket'].str.len()
    
    clean_data = dataframe.dropna(subset=['Fare'])
    
    sorted_by_name_length = clean_data.groupby('Ticket_name_length')['Fare'].mean().reset_index()
    
    graph = px.bar(sorted_by_name_length, x='Fare', y='Ticket_name_length', orientation='h',
                title='Average fare by ticket identifier length',
                labels={'Ticket_name_length':'Ticket identifier length', 'Fare':'Average fare'})
    graph.show()

horizontal_bar_chart(dataframe)

### 4.6 Pie chart

In [102]:
def pie_chart(dataframe: pd.DataFrame):
    fare_by_class = dataframe.groupby('Pclass', observed=True)['Fare'].sum().reset_index()
    graph = px.pie(fare_by_class, names='Pclass', values='Fare',
                labels={'Pclass':'Passenger class', 'Fare': 'Total fare'},
                title='Distribution of total fare by class')
    graph.show()

pie_chart(dataframe)

### 4.7 Second pie chart

In [103]:
def pie_chart_2(dataframe: pd.DataFrame):
    dataframe['Name_length'] = dataframe['Name'].str.len()
    graph = px.pie(dataframe, names='Sex', values='Name_length',
                labels={'Sex':'Passenger sex', 'Name_length':'Total length of full names'},
                title='Total length of full names distribution by sex')
    graph.show()

pie_chart_2(dataframe)

### 4.8 Box chart

In [104]:
def box_chart(dataframe: pd.DataFrame):
    graph = px.box(dataframe, x='Pclass', y='Age', color='Sex',
            labels={'Age': 'Passenger age', 'Pclass': 'Passenger class',
                'Sex': 'Passenger sex'},
            title='Distribution of age by class and gender')
    graph.show()

box_chart(dataframe)

### 4.9 Sunburst chart
#### 4.9.1 Заполнение пропусков в столбце 'Survived'
```python
clean_data = dataframe.copy()
clean_data["Survived"] = clean_data['Survived'].cat.add_categories('Missing')
clean_data['Survived'] = clean_data['Survived'].fillna('Missing')
clean_data['Survived'] = clean_data['Survived'].map({0: 'Died', 1: 'Survived', 'Missing': 'Missing'})
```

In [105]:
def sunburst_chart(dataframe: pd.DataFrame):
    clean_data = dataframe.copy()
    clean_data["Survived"] = clean_data['Survived'].cat.add_categories('Missing')
    clean_data['Survived'] = clean_data['Survived'].fillna('Missing')
    clean_data['Survived'] = clean_data['Survived'].map({0: 'Died', 1: 'Survived', 'Missing': 'Missing'})
    graph = px.sunburst(clean_data, path=['Pclass', 'Sex', 'Survived'],
                    labels={'Pclass':'Passenger class', 'Sex': 'Passenger sex'},
                    title='Sunburst chart: Passenger Distribution (Class-> Gender -> Survival)')
    graph.show()

sunburst_chart(dataframe)

### 4.10 3D scatter plot

In [106]:
def scatter_3d_plot(dataframe: pd.DataFrame):
    graph = px.scatter_3d(dataframe, x='Age', y='Fare', z='Pclass', color='Survived', 
                    symbol='Sex', hover_data=['Name'],
                    labels={'Pclass':'Passenger class', 'Sex':'Passenger sex', 'Fare':'Passenger fare',
                            'Age':'Passenger age'},
                    title='3D Scatter: Age-Fare-Class')
    graph.show()

scatter_3d_plot(dataframe)

## 5 Sankey diagram

### 4.1 Подготовка данных

In [107]:
def prepare_sankey_data(dataframe: pd.DataFrame):
    clean_data = dataframe.dropna(subset=['Pclass', 'Sex', 'Survived']).copy()
    sankey_data = clean_data.groupby(['Pclass', 'Sex', 'Survived'], observed=True).size().reset_index(name='count')
    return sankey_data

### 4.2 Создание узлов

In [108]:
def create_sankey_nodes(sankey_data: pd.DataFrame):
    class_nodes = [f'Class {pclass}' for pclass in sorted(sankey_data['Pclass'].unique())]
    sex_nodes = sorted(sankey_data['Sex'].unique())
    survival_nodes = ['Survived', 'Died']

    all_labels = class_nodes + sex_nodes + survival_nodes
    return all_labels, class_nodes, sex_nodes

### 4.3 Создание связей
#### 4.3.1 Создание связей Class -> Sex
```python
for i, pclass_str in enumerate(class_nodes):
        pclass_num = int(pclass_str.split()[-1])
        for j, sex in enumerate(sorted(sankey_data['Sex'].unique())):
            if (pclass_num, sex) in class_to_sex:
                flow_value = class_to_sex[(pclass_num, sex)]
                source.append(i)
                target.append(len(class_nodes) + j)
                value.append(flow_value)
```
где **i** - индекс **класса** (0 = Class 1, 1 = Class 2,...),  
**j** - индекс **пола** (0 = male, 1 = female),  
**len(class)nodes** - количество классов (3),  
len(class_nodes) + j = 3 + 0 = 3 (male), 3 + 1 = 4 (female),  
тогда получаем следующие связи:  
**source**: [0, 0, 1, 1, 2, 2] - индексы для **Class**,  
**target**: [3, 4, 3, 4, 3, 4] - индексы для **Sex** (оба из **labels**)  
**value**: [количество пассажиров для каждой комбинации]  
| **labels**  | Class 1 | Class 2 | Class 3 | male | female | Survived | Died |
|-----------|---------|---------|---------|------|--------|----------|------|
| **индексы** | 0       | 1       | 2       | 3    | 4      | 5        | 6    |
#### 4.3.2 Создание связей Sex -> Survival
```python
for j, sex in enumerate(sorted(sankey_data['Sex'].unique())):
    for k, survived in enumerate([0, 1]):
        if (sex, survived) in sex_to_survival:
            flow_value = sex_to_survival[(sex, survived)]
            source.append(len(class_nodes) + j)
            target.append(len(class_nodes) + len(sex_nodes) + k)
            value.append(flow_value)
```
аналогично: **j** - индекс **пола**,  
**k** - индекс **выживания** (0=Survived, 1=Died),  
**len(sex_nodes)** - количество полов (2),  
len(class_nodes) + j = 3 + 0 = 3 (male), 3 + 1 = 4 (female),  
len(class_nodes) + len(sex_nodes) + k = 3 + 2 + 0 = 5 (Survived), 3 + 2 + 1 = 6 (Died)

In [109]:
def create_sankey_links(sankey_data: pd.DataFrame, class_nodes, sex_nodes):
    source = []
    target = []
    value = []
    
    class_to_sex = sankey_data.groupby(['Pclass', 'Sex'], observed=True)['count'].sum()
    sex_to_survival = sankey_data.groupby(['Sex', 'Survived'], observed=True)['count'].sum()
    
    #class to sex nodes
    for i, pclass_str in enumerate(class_nodes):
        pclass_num = int(pclass_str.split()[-1])
        for j, sex in enumerate(sorted(sankey_data['Sex'].unique())):
            if (pclass_num, sex) in class_to_sex:
                flow_value = class_to_sex[(pclass_num, sex)]
                source.append(i)
                target.append(len(class_nodes) + j)
                value.append(flow_value)
    
    #sex to survived nodes 
    for j, sex in enumerate(sorted(sankey_data['Sex'].unique())):
        for k, survived in enumerate([0, 1]):
            if (sex, survived) in sex_to_survival:
                flow_value = sex_to_survival[(sex, survived)]
                source.append(len(class_nodes) + j)
                target.append(len(class_nodes) + len(sex_nodes) + k)
                value.append(flow_value)
    
    return source, target, value

### 4.4 Создание диаграммы

In [110]:
def create_sankey_diagram(labels, source, target, value):
    node_colors = [
    '#1E40AF', '#1D4ED8', '#3B82F6',
    '#EC4899', '#3B82F6',
    '#10B981', '#EF4444'
]

    graph = go.Figure(data=[go.Sankey(
        node = {
            'pad':15,
            'thickness':20,
            'line':{'color':'black', 'width':0.5},
            'label':labels,
            'color':node_colors
        },
        link = {
            'source':source,
            'target':target,
            'value':value,
            'color':"rgba(150, 150, 150, 0.4)"
        }
    )])

    graph.update_layout(
        title_text="Sankey diargam<br>Titanic passengers flow",
        font_size=12
    )
    
    return graph

### 4.5 Диаграмма

In [111]:
sankey_data = prepare_sankey_data(dataframe)
all_labels, class_nodes, sex_nodes= create_sankey_nodes(sankey_data)
source, target, value = create_sankey_links(sankey_data, class_nodes, sex_nodes)
graph = create_sankey_diagram(all_labels, source, target, value)
graph.show()